# Algoritmo Genético: De la Lógica Manual a la Implementación en Librerías
**Autor:** Bernardo Aedo

## 1. Implementación Manual de un Algoritmo Genético
Para comprender cómo funciona la optimización evolutiva, primero implementamos un algoritmo desde cero que busca optimizar los hiperparámetros de un `RandomForestClassifier`.

In [8]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Berna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
import random
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 1. Configuración del problema (Dataset Vinos)
data = load_wine()
X, y = data.data, data.target

# 2. Definición del "Individuo" (Genoma: n_estimators y max_depth)
def crear_individuo():
    return {
        'n_estimators': random.randint(10, 200),
        'max_depth': random.randint(1, 20)
    }

def crear_poblacion(tamano):
    return [crear_individuo() for _ in range(tamano)]

# 3. Función de Fitness (Aptitud = Accuracy del modelo)
def calcular_fitness(individuo):
    clf = RandomForestClassifier(
        n_estimators=individuo['n_estimators'],
        max_depth=individuo['max_depth'],
        random_state=42
    )
    scores = cross_val_score(clf, X, y, cv=5)
    return scores.mean()

# 4. Operadores Genéticos
def cruzar(padre1, padre2):
    # El hijo toma un gen de cada padre
    hijo = {
        'n_estimators': padre1['n_estimators'],
        'max_depth': padre2['max_depth']
    }
    return hijo

def mutar(individuo, probabilidad=0.1):
    hijo = individuo.copy()  # Crear copia
    if random.random() < probabilidad:
        hijo['n_estimators'] = random.randint(10, 200)
    if random.random() < probabilidad:
        hijo['max_depth'] = random.randint(1, 20)
    return hijo

# 5. Ejecución Evolutiva
def ejecutar_algoritmo_genetico(generaciones=5, tamano_poblacion=10):
    print(f"--- Iniciando Evolución Manual ---")
    poblacion = crear_poblacion(tamano_poblacion)
    
    for gen in range(generaciones):
        # Evaluar
        scores = [(ind, calcular_fitness(ind)) for ind in poblacion]
        scores.sort(key=lambda x: x[1], reverse=True)
        
        mejor_score = scores[0][1]
        print(f"Generación {gen+1}: Mejor Accuracy = {mejor_score:.4f}")
        
        # Selección (Elitismo)
        padres = [scores[0][0], scores[1][0]]
        
        # Reproducción
        nueva_poblacion = padres[:]
        while len(nueva_poblacion) < tamano_poblacion:
            hijo = cruzar(padres[0], padres[1])
            hijo = mutar(hijo)
            nueva_poblacion.append(hijo)
        poblacion = nueva_poblacion

ejecutar_algoritmo_genetico()

--- Iniciando Evolución Manual ---
Generación 1: Mejor Accuracy = 0.9778
Generación 2: Mejor Accuracy = 0.9778
Generación 3: Mejor Accuracy = 0.9778
Generación 4: Mejor Accuracy = 0.9778
Generación 5: Mejor Accuracy = 0.9778


## 2. Conexión con GASearchCV (sklearn-genetic-opt)

El código anterior muestra la "mecánica interna" de la evolución. La herramienta profesional `GASearchCV` que se utiliza en la industria funciona bajo los mismos principios biológicos, pero automatizados.

### Comparativa: Manual vs Librería

| Concepto Biológico | En nuestro código manual | En GASearchCV |
| :--- | :--- | :--- |
| **Individuo** | Diccionario `{'n_estimators': 100, ...}` | Un estimador de Scikit-Learn configurado. |
| **Población** | Lista de diccionarios. | Parámetro `population_size=10`. |
| **Fitness (Aptitud)** | Función `calcular_fitness()` usando `cross_val_score`. | Parámetro `scoring='accuracy'` y validación cruzada interna `cv`. |
| **Genes (Espacio de búsqueda)** | `random.randint(10, 200)` | `param_grid` usando `Integer(10, 200)` o `Categorical`. |
| **Evolución** | Funciones `cruzar()` y `mutar()` dentro de un bucle `for`. | Motor interno `algorithm='eaMuPlusLambda'` controlado por `crossover_probability` y `mutation_probability`. |

### Implementación con Librería (`sklearn-genetic-opt`)
Para contrastar nuestro algoritmo manual, a continuación ejecutamos el código estándar enseñado en clases para esta librería.

> *Fuente: Extracto del material académico "Clase 17".*

#### Sintaxis de Implementación en GASearchCV
Así es como se configura el estimador usando la librería (ejemplo ilustrativo):

```python
# Cita: Código extraído del material docente "clase_17.ipynb"
evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=10,
                               generations=35,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param_grid,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

### Comparativa de Parámetros: Manual vs GASearchCV

A continuación, contrastamos cómo definimos los parámetros en nuestra función manual `ejecutar_algoritmo_genetico` versus cómo los solicita la librería profesional:

| Concepto | En nuestro código Manual | En GASearchCV (Clase 17) |
| :--- | :--- | :--- |
| **Población** | `tamano_poblacion=10` | `population_size=10` |
| **Duración** | `generations=5` | `generations=35` |
| **Evaluación** | `calcular_fitness()` | `scoring='accuracy'` |
| **Cruce** | Función `cruzar()` (100% implícito) | `crossover_probability=0.8` |
| **Mutación** | Función `mutar()` (`probabilidad=0.1`) | `mutation_probability=0.1` |
| **Elitismo** | Selección de los 2 mejores padres | `elitism=True` |

> **Observación:** La librería nos permite controlar con mayor granularidad probabilidades que en nuestro código manual estaban fijas o simplificadas.

**Conclusión:** `GASearchCV` no es una "caja negra"; es un gestor de poblaciones que toma hiperparámetros, los cruza y los muta iterativamente para maximizar el `scoring` definido, ahorrándonos la escritura manual de los operadores genéticos.